In [1]:
import cirq
import qibo
qibo.set_backend("numpy")

import qibo.backends
import tequila as tq
import numpy as np
import copy
import sys
sys.path.append('../../Gates_Lab_Suite')

from Auto_Algorithm import *
from Core_Definition import *
from Visualization import *

from circuit_tools import (
    convert_gate_cirq_to_gates,
    cirq_to_gates, 
    cirq_to_qibo,
    reduce_measurements_naive,
    measurment_gate_qibo,
    measurment_gate_gates,
    zero_state,
    get_results_from_frequencies,
    get_energy_from_data)

import time

import matplotlib.pyplot as plt

from numbers import Number

[Qibo 0.2.18|INFO|2025-07-10 10:05:37]: Using numpy backend on /CPU:0
/home/liamj/Molecular Simulation/chemical-graph/tequila-beta/lib/python3.12/site-packages/tequila/simulators/simulator_api.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
np.__version__

'1.26.4'

# Jordan Wigner Transformation

In [3]:
# Define the geometry
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

# Get molecule
mol_JW = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="JordanWigner").use_native_orbitals()

In [4]:
print('HF ', mol_JW.compute_energy('HF'))
fci = mol_JW.compute_energy('FCI')
print('FCI ', fci)

/home/liamj/Molecular Simulation/chemical-graph/tequila-beta/lib/python3.12/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)


HF  -1.8291374124430182
FCI  -1.9961503255188089


### Method 1: G1 SPA

In [5]:
U_JW_SPA = mol_JW.make_ansatz("SPA", edges=[(0,1),(2,3)])
H_JW_G1 = mol_JW.make_hamiltonian()

# IS THIS G1?
# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol_JW.UR(0,1,'a') + mol_JW.UR(2,3,'b')
U_JW_G1 = U0 + U_JW_SPA + U0.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1,U=U_JW_G1),silent=True)

U_JW_G1_mapped = U_JW_G1.map_variables(variables=res.variables)

for gate in U_JW_G1_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G1 = tq.compile(U_JW_G1_mapped, backend="cirq")

energy_JW_G1 = res.energy
print(f"difference from fci: {abs(res.energy-fci)*1000} meh")

print(f"Original number of measurements: {len(H_JW_G1.keys())}")
reduced_measurements_JW_G1 = reduce_measurements_naive(H_JW_G1, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G1)}")

No variables given...
difference from fci: 39.8957042850312 meh
Original number of measurements: 361
Reduced number of measurements: 103


### Method 2: G1 SPA with optimized orbitals

In [6]:
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol_JW,circuit=U_JW_SPA, initial_guess=guess, silent=True).molecule
H_JW_G1_optimized_orbitals = opt.make_hamiltonian()
cirq_JW_G1_optimized_orbitals = tq.compile(U_JW_SPA, backend="cirq")
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1_optimized_orbitals,U=U_JW_SPA), silent=True)

U_JW_G1_optimized_orbitals_fixed_params = U_JW_SPA.map_variables(variables=res.variables)
cirq_JW_G1_optimized_orbitals_fixed_params = tq.compile(U_JW_G1_optimized_orbitals_fixed_params, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
energy_JW_G1_optimized_orbitals_fixed_params = res.energy

print(f"Original number of measurements: {len(H_JW_G1_optimized_orbitals.keys())}")
reduced_measurements_JW_G1_optimized_orbitals = reduce_measurements_naive(H_JW_G1_optimized_orbitals, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G1_optimized_orbitals)}")

No variables given...
No variables given...
difference from fci: 16.262440975463257 meh?

Original number of measurements: 361
Reduced number of measurements: 103


### Method 3: G2

In [7]:
# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol_JW.UR(1,2,'c')
UC = mol_JW.UC(1,2,'d')
U_JW_G2 = U_JW_SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1,U=U_JW_G2), silent=True)

U_JW_G2_mapped = U_JW_G2.map_variables(variables=res.variables)

for gate in U_JW_G2_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G2 = tq.compile(U_JW_G2_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")
energy_JW_G2 = res.energy

print(f"Original number of measurements: {len(H_JW_G1.keys())}")
reduced_measurements_JW_G2 = reduce_measurements_naive(H_JW_G1, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G2)}")

No variables given...
difference from fci: 16.69940439791273 meh?

Original number of measurements: 361
Reduced number of measurements: 103


### Method 4: G2 with optimized Orbitals

In [8]:
U_JW_G2_optimized_orbitals = U_JW_SPA
U_JW_G2_optimized_orbitals += mol_JW.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol_JW.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_JW_G2_optimized_orbitals += mol_JW.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol_JW.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_JW_G2_optimized_orbitals += mol_JW.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol_JW.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_JW_G1_optimized_orbitals,U=U_JW_G2_optimized_orbitals),silent=True)

U_JW_G2_optimized_orbitals_mapped = U_JW_G2_optimized_orbitals.map_variables(variables=res.variables)

for gate in U_JW_G2_optimized_orbitals_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_JW_G2_optimized_orbitals = tq.compile(U_JW_G2_optimized_orbitals_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")
energy_JW_G2_optimized_orbitals = res.energy

print(f"Original number of measurements: {len(H_JW_G1_optimized_orbitals.keys())}")
reduced_measurements_JW_G2_optimized_orbitals = reduce_measurements_naive(H_JW_G1_optimized_orbitals, 8)
print(f"Reduced number of measurements: {len(reduced_measurements_JW_G2_optimized_orbitals)}")

No variables given...
difference from fci: 9.697128646922515 meh?
Original number of measurements: 361
Reduced number of measurements: 103


# Parity Tapering Transformation

In [9]:
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", transformation="TaperedBinary").use_native_orbitals()

In [10]:
# You can check the Hartree Fock Energy and FCI energy like this:
print('HF ', mol.compute_energy('HF'))
fci = mol.compute_energy('FCI')
print('FCI ', fci)

HF  -1.8291374124430178
FCI  -1.9961503255188089


### Method 1: G1

In [11]:
U_PT_SPA = mol.make_ansatz("SPA", edges=[(0,1),(2,3)])
H_PT_G1 = mol.make_hamiltonian()

# This is how to properly calculate the SPA energy, without optimizing the orbitals
# The UR0 basically optimizes the orbitals
U0 = mol.UR(0,1,'a') + mol.UR(2,3,'b')
U_PT_G1 = U0 + U_PT_SPA + U0.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1,U=U_PT_G1),silent=True)

U_PT_G1_mapped = U_PT_G1.map_variables(variables=res.variables)

for gate in U_PT_G1_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G1 = tq.compile(U_PT_G1_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?")

print(f"Original number of measurements: {len(H_PT_G1.keys())}")
reduced_measurements_PT_G1 = reduce_measurements_naive(H_PT_G1, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G1)}")

No variables given...
difference from fci: 39.895704285025204 meh?
Original number of measurements: 317
Reduced number of measurements: 88


### Method 2: G1 with optimized Orbitals

In [12]:
# If u want regular obrital optimization u can use:
guess = np.eye(4)
opt = tq.quantumchemistry.optimize_orbitals(molecule=mol,circuit=U_PT_SPA, initial_guess=guess, silent=True).molecule
H_PT_G1_opt = opt.make_hamiltonian()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1_opt,U=U_PT_SPA), silent=True)

for gate in U_JW_G1_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G1_optimized_orbitals = tq.compile(U_PT_SPA, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1_opt.keys())}")
reduced_measurements_PT_G1_optimized_orbitals = reduce_measurements_naive(H_PT_G1_opt, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G1_optimized_orbitals)}")

No variables given...
difference from fci: 16.26244097546392 meh?

Original number of measurements: 325
Reduced number of measurements: 84


### Method 3: G2

In [13]:
# If u want to use Orbital Correlator for more graphs eg. (1,2) u can use:
U1 = mol.UR(1,2,'c')
UC = mol.UC(1,2,'d')
U_PT_G2 = U_PT_SPA + U0 + U1 + UC + U0.dagger() + U1.dagger()
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1,U=U_PT_G2), silent=True)

U_PT_G2_mapped = U_PT_G2.map_variables(variables=res.variables)

for gate in U_PT_G2_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G2 = tq.compile(U_PT_G2_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1.keys())}")
reduced_measurements_PT_G2 = reduce_measurements_naive(H_PT_G1, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G2)}")

No variables given...
difference from fci: 16.69940439739115 meh?

Original number of measurements: 317
Reduced number of measurements: 88


### Method 4: G2 with Optimized Orbitals

In [14]:
# U can also add Orbital optimization to the Rotator - Corellator / use our optimized Hamiltonian
U_PT_G2_optimized_orbitals = U_PT_SPA
U_PT_G2_optimized_orbitals += mol.UR(0,2, angle=(tq.Variable("a_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("a_2") + 0.5) * np.pi)
U_PT_G2_optimized_orbitals += mol.UC(0,2, angle=tq.Variable("b_1") * np.pi) + mol.UC(1,3, angle=tq.Variable("b_2") * np.pi)
U_PT_G2_optimized_orbitals += mol.UR(0,2, angle=(tq.Variable("c_1") + 0.5) * np.pi) + mol.UR(1,3, angle=(tq.Variable("c_2") + 0.5) * np.pi)
res = tq.minimize(tq.ExpectationValue(H=H_PT_G1_opt,U=U_PT_G2_optimized_orbitals),silent=True)

U_PT_G2_optimized_orbitals_mapped = U_PT_G2_optimized_orbitals.map_variables(variables=res.variables)

for gate in U_PT_G2_optimized_orbitals_mapped.gates:
    try:
        if not isinstance(gate.parameter, tq.objective.objective.Variable):
            gate.parameter = gate.parameter.transformation(gate.parameter.args[0])
    except:
        x = 1

cirq_PT_G2_optimized_orbitals = tq.compile(U_PT_G2_optimized_orbitals_mapped, backend="cirq")

print(f"difference from fci: {abs(res.energy-fci)*1000} meh?\n")

print(f"Original number of measurements: {len(H_PT_G1_opt.keys())}")
reduced_measurements_PT_G2_optimized_orbitals = reduce_measurements_naive(H_PT_G1_opt, 6)
print(f"Reduced number of measurements: {len(reduced_measurements_PT_G2_optimized_orbitals)}")

No variables given...
difference from fci: 9.697128646919184 meh?

Original number of measurements: 325
Reduced number of measurements: 84


# Get all circuits as Gates Lab Suite circuits (for taking data)

In [15]:

# Parameterized circuits
gates_JW_G1 = cirq_to_gates(cirq_JW_G1.circuit, 8, "Jordan-Wigner G1")
gates_JW_G1_optimized_orbitals = cirq_to_gates(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")
gates_JW_G2 = cirq_to_gates(cirq_JW_G2.circuit, 8, "Jordan-Wigner G2")
gates_JW_G2_optimized_orbitals = cirq_to_gates(cirq_JW_G2_optimized_orbitals.circuit, 8, "Jordan-Wigner G2 w/ optimized orbitals")

gates_PT_G1 = cirq_to_gates(cirq_PT_G1.circuit, 8, "Parity Tapering G1")
gates_PT_G1_optimized_orbitals = cirq_to_gates(cirq_PT_G1_optimized_orbitals.circuit, 8, "Parity Tapering G1 w/ optimized orbitals")
gates_PT_G2 = cirq_to_gates(cirq_PT_G2.circuit, 8, "Parity Tapering G2")
gates_PT_G2_optimized_orbitals = cirq_to_gates(cirq_PT_G2_optimized_orbitals.circuit, 8, "Parity Tapering G2 w/ optimized orbitals")


# Circuits with fixed, optimal parameters
gates_JW_G1_optimized_orbitals_fixed_params = cirq_to_gates(cirq_JW_G1_optimized_orbitals_fixed_params.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")

In [16]:
res = np.zeros((len(reduced_measurements_JW_G1), 2 ** 8))

for i in range(len(reduced_measurements_JW_G1)):

    print(i, end="\r")

    gates_JW_G1_copy = copy.deepcopy(gates_JW_G1)

    for j in range(8):
        pauli_char = list(reduced_measurements_JW_G1.keys())[i][j]
        if pauli_char != "Z":
            gates_JW_G1_copy.Add_Gate(measurment_gate_gates(pauli_char, j))

    # final_state = gates_JW_G1_copy.Simulate()
    # res[i] = np.array(final_state.population)

    seq = gates_JW_G1_copy.GatesLab_Sequence()

In [17]:
for moment in cirq_JW_G2.circuit:
    for op in moment:
        if isinstance(op.gate, cirq.ControlledGate):
            print(op.qubits[0])

q(3)


# Get all circuits as Qibo circuits (for simulations)

In [18]:

# Parameterized circuits
qibo_JW_G1 = cirq_to_qibo(cirq_JW_G1.circuit, 8, "Jordan-Wigner G1")
qibo_JW_G1_optimized_orbitals = cirq_to_qibo(cirq_JW_G1_optimized_orbitals.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")
qibo_JW_G2 = cirq_to_qibo(cirq_JW_G2.circuit, 8, "Jordan-Wigner G2")
qibo_JW_G2_optimized_orbitals = cirq_to_qibo(cirq_JW_G2_optimized_orbitals.circuit, 8, "Jordan-Wigner G2 w/ optimized orbitals")

qibo_PT_G1 = cirq_to_qibo(cirq_PT_G1.circuit, 8, "Parity Tapering G1")
qibo_PT_G1_optimized_orbitals = cirq_to_qibo(cirq_PT_G1_optimized_orbitals.circuit, 8, "Parity Tapering G1 w/ optimized orbitals")
qibo_PT_G2 = cirq_to_qibo(cirq_PT_G2.circuit, 8, "Parity Tapering G2")
qibo_PT_G2_optimized_orbitals = cirq_to_qibo(cirq_PT_G2_optimized_orbitals.circuit, 8, "Parity Tapering G2 w/ optimized orbitals")

# Circuits with fixed, optimal parameters
qibo_JW_G1_optimized_orbitals_fixed_params = cirq_to_qibo(cirq_JW_G1_optimized_orbitals_fixed_params.circuit, 8, "Jordan-Wigner G1 w/ optimized orbitals")


In [19]:
nqubits = 8

initial_rho = zero_state(nqubits)
nshots = 5000

res = []
res_sn = []

data_dict = {}
data_dict_sn = {}

for i in range(len(reduced_measurements_JW_G1)):

    print(i, end="\r")

    qibo_JW_G1_copy = copy.deepcopy(qibo_JW_G1)

    for j in range(nqubits):
        pauli_char = list(reduced_measurements_JW_G1.keys())[i][j]
        if pauli_char != "Z":
            qibo_JW_G1_copy.add(measurment_gate_qibo(pauli_char, j))

    qibo_JW_G1_copy.add(qibo.gates.M(*range(nqubits)))

    result_sn = qibo_JW_G1_copy(nshots=nshots, initial_state=initial_rho)
    result = qibo_JW_G1_copy(nshots=1, initial_state=initial_rho)

    res.append(result.probabilities())
    data_dict[list(reduced_measurements_JW_G1.keys())[i]] = result.probabilities()

    result_shotnoise = get_results_from_frequencies(result_sn.frequencies(), nshots, nqubits)
    res_sn.append(result_shotnoise)
    
    data_dict_sn[list(reduced_measurements_JW_G1.keys())[i]] = result_shotnoise

In [20]:
energy_JW_G2

-1.9794509211208962

In [21]:
energy_JW_G2_optimized_orbitals

-1.9864531968718864

In [22]:
energy_JW_G1_simulated = get_energy_from_data(H_JW_G1, data_dict, reduced_measurements_JW_G1)

In [23]:
energy_JW_G1

-1.9562546212337777

In [24]:
energy_JW_G1_simulated

1.3271363511128893